<a href="https://colab.research.google.com/github/rochefort8/AnalogCounter-NN/blob/master/acr_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 新しいセクション

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!rm -rf /var/test
!rm -rf /var/input

!tar zxpf  "/content/drive/My Drive/AC/input.tar.gz" -C /var 
!tar zxpf  "/content/drive/My Drive/AC/test.tar.gz" -C /var 
#!tar zxpf /var/input.tar.gz -C /var
#!tar zxpf /var/test.tar.gz -C /var

In [0]:
#!rm /var/input.tar.gz

In [0]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from PIL import Image
from io import BytesIO 

import cv2 as cv

import matplotlib.pyplot as plt
import numpy as np
import os

from keras.preprocessing.image import array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split

batch_size = 128
num_classes = 20
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

In [0]:

# the data, split between train and test sets
#(x_train, y_train), (x_test, y_test) = mnist.load_data()
# print(type(y_train))

In [0]:
X=[]
Y=[]

for number in range (10):  
  path = "/var/input/" + str(number) + "/"
  files = os.listdir(path)
  print(number)
  for file in files:
      img = np.asarray(cv.imread(path + file,0))

#      threshold = 66
#      ret,img = cv.threshold(img,threshold,255,cv.THRESH_TOZERO)
#      threshold = 100
#      ret,img = cv.threshold(img,threshold,255,cv.THRESH_BINARY)

      X.append(img)
      Y.append(number)

np.random.seed(1)
np.random.shuffle(X)
np.random.seed(1)
np.random.shuffle(Y)

#x_train = np.array(X)
#y_train = np.array(Y)
_x_train, _x_test, _y_train, _y_test =train_test_split(X, Y, test_size=0.33, random_state=111)
x_train = np.array(_x_train)
x_test = np.array(_x_test)
y_train = np.array(_y_train)
y_test = np.array(_y_test)

idx = 704
plt.imshow(x_train[idx])
print(y_train[idx])

#(x_train, y_train), (x_test, y_test) = mnist.load_data()
XX=[]
YY=[]

for number in range (10):  
  path = "/var/test/" + "{:02d}".format(number) + "/"
  files = os.listdir(path)
  del XX[:]
  del YY[:]
  for file in files:
#      im2 = Image.open(path + file)
      img = np.asarray(cv.imread(path + file,0))
      threshold = 66
      ret,img = cv.threshold(img,threshold,255,cv.THRESH_TOZERO)
#      threshold = 100
#      ret,img = cv.threshold(img,threshold,255,cv.THRESH_BINARY)
      XX.append(img)
      YY.append(number)

np.random.seed(1)
np.random.shuffle(XX)
np.random.seed(1)
np.random.shuffle(YY)

#x_test = np.array(XX)
#y_test = np.array(YY)
#(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
model.save('/var/tmp/ac')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print(score)

In [0]:
#(x_train, y_train), (x_test, y_test) = mnist.load_data()
XX=[]
YY=[]

for number in range (10):  
  path = "/var/test/" + "{:02d}".format(number) + "/"
  files = os.listdir(path)
  del XX[:]
  del YY[:]

  for file in files:
#      im2 = Image.open(path + file)

      img = np.asarray(cv.imread(path + file,0))
      threshold = 66
      ret,img = cv.threshold(img,threshold,255,cv.THRESH_TOZERO)
#      threshold = 100
#      ret,img = cv.threshold(img,threshold,255,cv.THRESH_TOZERO)

      XX.append(img)
      YY.append(number)

  x_data = np.array(XX) 
  y_data = np.array(YY)

  if K.image_data_format() == 'channels_first':
    x_data = x_data.reshape(x_data.shape[0], 1, img_rows, img_cols)
  else:
    x_data = x_data.reshape(x_data.shape[0], img_rows, img_cols, 1)

  x_data = x_data.astype('float32')
  x_data /= 255
  y_data = keras.utils.to_categorical(y_data, num_classes)
  score = model.evaluate(x_data, y_data, verbose=0)
  print('Number :' +str(number))
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])
  expect = model.predict_classes(x_data,batch_size=5)
  print(expect)

del XX[:]
del YY[:]

for number in range (10):  
  path = "/var/test/" + "{:02d}".format(number) + "/"
  files = os.listdir(path)
  for file in files:
      im2 = Image.open(path + file)
      img = np.asarray(cv.imread(path + file,0).reshape(28,28))

      threshold = 66
      ret,img = cv.threshold(img,threshold,255,cv.THRESH_TOZERO)
#      threshold = 100
#      ret,img = cv.threshold(img,threshold,255,cv.THRESH_TOZERO)
#      ret,img = cv.threshold(img,threshold,255,cv.THRESH_BINARY)
      XX.append(img)
      YY.append(number)

np.random.seed(1)
np.random.shuffle(XX)
np.random.seed(1)
np.random.shuffle(YY)

plt.imshow(XX[0])

x_data = np.array(XX)
y_data = np.array(YY)


if K.image_data_format() == 'channels_first':
    x_data = x_data.reshape(x_data.shape[0], 1, img_rows, img_cols)
else:
    x_data = x_data.reshape(x_data.shape[0], img_rows, img_cols, 1)

x_data = x_data.astype('float32')
x_data /= 255
y_data = keras.utils.to_categorical(y_data, num_classes)
score = model.evaluate(x_data, y_data, verbose=0)
print('=== Total ====')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

